In [ ]:
"""Combining Front Office and Middle Office

Demonstration of how to give front office users an view of intraday trading activity on top of their middle office IBOR.

Attributes
----------
instruments
transactions
sub-holdings keys
properties
orders
derived portfolios
cocoon
"""

# Combining Front Office and Middle Office

In this example we demonstrate how to give front office users a view of intraday trading activity on top of their middle office IBOR. We set up a portfolio with multiple strategies and generate orders which are booked back into a [derived portfolio](https://support.finbourne.com/what-is-a-derived-portfolio) showing updated positions in the strategies.

To illustrate, below we will BUY and SELL <i>Amazon, Inc.</i> stock across two strategies, and show an updated view of the portfolio intraday. Consider the <b>Tech</b> strategy. At SOD we have 5000 units, we SELL 500 units, leaving 4500 in the portfolio. 

<br>


|Portfolio Code | Instrument Name |ISIN| Strategy  | SOD Holdings | Intraday orders | Intraday Holdings|
|:-------------:| :-------------: | :-----------------: |:-------------: | :-------------: | :-------------: | :-------------: |
|us_long_live| Amazon |US0231351067 |Balanced | 0 | BUY of 4000 units | <font color="red">4000 </font>|
|us_long_live| Amazon |US0231351067|Tech | 5000 | SELL of 500 units | <font color="red">4500 </font>|

<br>

## Imports

In [1]:
import lusid
import lusid.models as models
import lusid.api as la
import lusid.models as lm
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.0.1.0


Define scope, portfolio and some other variables used in the example:

In [2]:
orders_scope = "combining_fo_and_mo_scope"
orders_portfolio = "us_long_live"
ibor_scope = "middle_office"
ibor_portfolio = "us_long"
strategy_shk = "strategy"
executed_price_code = "executed_price"
execution_price_property = f"Allocation/{orders_scope}/{executed_price_code}"
allocation_strategy_property = f"Allocation/{orders_scope}/{strategy_shk}"
pf_created_date = "2020-01-01T00:00:00+00:00"

## Load an Instrument Master into LUSID

In [3]:
instr_df = pd.read_csv("data/multiplecurrencies-instruments.csv")
display(instr_df)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,BBG00FN3B5K8,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.000
3,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,BBG000DQQNJ8,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.875
4,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,NaN
5,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,NaN
6,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,NaN
7,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,NaN
8,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,NaN


In [4]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}

In [5]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=ibor_scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,9,0,0


## Create LUSID Properties

Create the properties for the sub-holding keys and executed price on the allocations

In [6]:
properties_api = api_factory.build(la.PropertyDefinitionsApi)

def create_property_definition(domain, scope, code, data_type):
    try:
        properties_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=code,
                display_name=code,
                life_time="Perpetual",
                value_required=False,
                data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type)
            )
        )
    except ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 124:  # 'PropertyAlreadyExists'
            raise e

In [7]:
# sub-holding keys
create_property_definition("Transaction", ibor_scope, strategy_shk, "string")

# allocation execution price
create_property_definition("Allocation", orders_scope, executed_price_code, "number")

# allocation strategy name
create_property_definition("Allocation", orders_scope, strategy_shk, "string")

## Upload a Portfolio into LUSID

In [8]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code, "portfolio_name": name},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {
            "created": f"${pf_created_date}"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[strategy_shk],
    )

    succ, failed = format_portfolios_response(result)
    display(failed)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])) 

Create the main portfolio

In [9]:
create_portfolio(ibor_scope, ibor_portfolio, "ibor")

,error_items,status


,success,failed,errors
0,1,0,0


## Create a Derived portfolio to hold intraday activity

Create a derived portfolio to capture the executed orders to give us a front office live postion view

In [10]:
derivedpf_api = la.DerivedTransactionPortfoliosApi(api_factory.build(la.DerivedTransactionPortfoliosApi))

try:
    dp_request = lm.CreateDerivedTransactionPortfolioRequest(
        code="us_long_live",
        display_name="us_long_live",
        parent_portfolio_id=lm.ResourceId(ibor_scope, ibor_portfolio),
        created=pf_created_date,    
    )

    dp_result = derivedpf_api.create_derived_portfolio(scope=ibor_scope, create_derived_transaction_portfolio_request=dp_request)
except ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 112:  # 'PortfolioWithIdAlreadyExists'
        raise e

## Set initial holdings in the parent portfolio

In [11]:
hldgs_df = pd.read_csv("data/initial_holdings.csv")
display(hldgs_df)

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency,strategy
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,1550.00,7750000.00,USD,Tech
1,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,49567,190.00,9417730.00,USD,Tech
2,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,98444,140.98,13878635.12,USD,Balanced
3,Apple_Nasdaq_AAPL,imd_35345375,US0378331005,BBG000B9XVV8,350000,190.00,9417730.00,USD,Balanced
4,USTreasury_2.00_2021,imd_34535363,US912828U816,BBG00FN3B5K8,1433,99.25,12063142.75,USD,Balanced


In [12]:
holdings_mapping = {
    "required":{
        "code": f"${ibor_portfolio}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity"
    },
    "identifier_mapping": {
        "Figi": "figi"
    },
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost"
    }
}

In [13]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=ibor_scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    sub_holding_keys=[strategy_shk],
    file_type="holdings"
)

succ, failed = format_holdings_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


## Check the initial portfolio holdings

In [14]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=ibor_scope, code=ibor_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True).sort_values(by='strategy(middle_office-SubHoldingKeys)')

,instrument_uid,strategy(middle_office-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency
2,LUID_00003D4Y,Balanced,Apple_Nasdaq_AAPL,us_long,middle_office,P,350000.0,350000.0,9417730.00,USD,0.0,USD,USD
3,LUID_00003D4X,Balanced,USTreasury_6.875_2025,us_long,middle_office,P,98444.0,98444.0,13878635.12,USD,0.0,USD,USD
4,LUID_00003D51,Balanced,USTreasury_2.00_2021,us_long,middle_office,P,1433.0,1433.0,12063142.75,USD,0.0,USD,USD
0,LUID_00003D4Z,Tech,Amazon_Nasdaq_AMZN,us_long,middle_office,P,5000.0,5000.0,7750000.00,USD,0.0,USD,USD
1,LUID_00003D4Y,Tech,Apple_Nasdaq_AAPL,us_long,middle_office,P,49567.0,49567.0,9417730.00,USD,0.0,USD,USD


## Post orders into LUSID

In this section, we post some [orders](https://support.finbourne.com/how-does-lusid-support-the-trade-lifecycle) into LUSID. 

In [16]:
orders_df = pd.read_csv('data/orders.csv')
orders_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side,strategy
0,us_long,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,99.25,USD,ORD001,buy,Balanced
1,us_long,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121,99.30,USD,ORD001,buy,Balanced
2,us_long,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,500,1400.00,USD,ORD002,sell,Tech
3,us_long,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,4000,1550.00,USD,ORD003,buy,Balanced
4,us_long,USTreasury_6.875_2025,imd_34534512,US912810EV62,BBG000DQQNJ8,5643,140.98,USD,ORD004,buy,Balanced
5,us_long,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,32000,150.00,USD,ORD005,sell,Balanced


In [19]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)

for index, order in orders_df.iterrows():
    
    portfolio = order['portfolio']

    order_requests[portfolio].append(
        models.OrderRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi']
            },
            properties={},
            portfolio_id=models.ResourceId(
                scope=ibor_scope,
                code=ibor_portfolio
            ),
            state="new",
            type="limit",
            price=models.CurrencyAndAmount(
                        amount=order['quantity'] * order['price'],
                        currency=order['currency'])
        )
    )

for order_portfolio in order_requests:
    order_sets[order_portfolio] = models.OrderSetRequest(
        order_requests=order_requests[order_portfolio]
    )    

for order_portfolio in order_sets:    
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=order_sets[order_portfolio]
    )
    
    display(lusid_response_to_data_frame(response))

ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 30 Sep 2021 21:03:52 GMT', 'Content-Type': 'application/problem+json', 'Server': 'Kestrel', 'Transfer-Encoding': 'chunked', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HMC4AGELOBEU:00000001', 'lusid-meta-correlationId': '0HMC4AGELOBEU:00000001', 'lusid-meta-duration': '36'})
HTTP response body: {"name":"UnknownError","errorDetails":[],"code":-1,"type":"https://docs.lusid.com/#section/Error-Codes/-1","title":"An unexpected problem has occurred - DEBUG INFO: There was a problem with the request","status":500,"detail":"An unknown problem occurred on our side whilst attempting to execute request. This shouldn't happen and it will be flagged for attention by us, but if it persists please get in contact with our support team. - DEBUG INFO: There was a problem with one or more of the fields in the request - System.ArgumentException: An element with the same key but a different v

## Book Allocations against the orders

[Allocations](https://support.finbourne.com/what-is-an-allocation-in-lusid) are then booked off the back of these orders.

In [61]:
allocations_df = pd.read_csv('data/allocations.csv')
allocations_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,allocation_id,originating_order,strategy
0,us_long,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,80,USD,ALLOC001,ORD001,Balanced
1,us_long,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,500,1600,USD,ALLOC002,ORD002,Tech
2,us_long,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,4000,1600,USD,ALLOC003,ORD003,Balanced
3,us_long,USTreasury_6.875_2025,imd_34534512,US912810EV62,BBG000DQQNJ8,5643,150,USD,ALLOC004,ORD004,Balanced
4,us_long,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,32000,150,USD,ALLOC005,ORD005,Balanced


In [63]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    allocation_requests[portfolio].append(
        models.AllocationRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id=models.ResourceId(
                scope=orders_scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            instrument_identifiers={
                'Instrument/default/Figi': allocation['figi']
            },
            properties={
                execution_price_property: lm.PerpetualProperty(
                    key=execution_price_property,
                    value=lm.PropertyValue(metric_value=lm.MetricValue(value=allocation["price"]))
                ),
                allocation_strategy_property: lm.PerpetualProperty(
                    key=allocation_strategy_property,
                    value=lm.PropertyValue(label_value=allocation['strategy'])
                )
            },
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=orders_portfolio
            )
        )
    )
    
for allocation_portfolio in allocation_requests:    
    allocation_sets[allocation_portfolio] = models.AllocationSetRequest(
        allocation_requests=allocation_requests[allocation_portfolio]
    )

for allocation_portfolio in allocation_sets:
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        allocation_set_request=allocation_sets[allocation_portfolio]
    )
    
    display(lusid_response_to_data_frame(response))

,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/Figi,version.effective_from,version.as_at_date,properties.Allocation/combining_fo_and_mo_scope/strategy.key,properties.Allocation/combining_fo_and_mo_scope/strategy.value.label_value,properties.Allocation/combining_fo_and_mo_scope/executed_price.key,properties.Allocation/combining_fo_and_mo_scope/executed_price.value.metric_value.value,lusid_instrument_id
0,combining_fo_and_mo_scope,ALLOC004,combining_fo_and_mo_scope,ORD004,combining_fo_and_mo_scope,us_long_live,5643,BBG000DQQNJ8,0001-01-01 00:00:00+00:00,2021-09-30 16:49:03.375308+00:00,Allocation/combining_fo_and_mo_scope/strategy,Balanced,Allocation/combining_fo_and_mo_scope/executed_...,150.0,LUID_00003D53
1,combining_fo_and_mo_scope,ALLOC003,combining_fo_and_mo_scope,ORD003,combining_fo_and_mo_scope,us_long_live,4000,BBG000BVPXP1,0001-01-01 00:00:00+00:00,2021-09-30 16:49:03.375308+00:00,Allocation/combining_fo_and_mo_scope/strategy,Balanced,Allocation/combining_fo_and_mo_scope/executed_...,1600.0,LUID_00003D52
2,combining_fo_and_mo_scope,ALLOC001,combining_fo_and_mo_scope,ORD001,combining_fo_and_mo_scope,us_long_live,121543,BBG00FN3B5K8,0001-01-01 00:00:00+00:00,2021-09-30 16:49:03.375308+00:00,Allocation/combining_fo_and_mo_scope/strategy,Balanced,Allocation/combining_fo_and_mo_scope/executed_...,80.0,LUID_00003D4Z
3,combining_fo_and_mo_scope,ALLOC002,combining_fo_and_mo_scope,ORD002,combining_fo_and_mo_scope,us_long_live,500,BBG000BVPXP1,0001-01-01 00:00:00+00:00,2021-09-30 16:49:03.375308+00:00,Allocation/combining_fo_and_mo_scope/strategy,Tech,Allocation/combining_fo_and_mo_scope/executed_...,1600.0,LUID_00003D52
4,combining_fo_and_mo_scope,ALLOC005,combining_fo_and_mo_scope,ORD005,combining_fo_and_mo_scope,us_long_live,32000,BBG000DQQNJ8,0001-01-01 00:00:00+00:00,2021-09-30 16:49:03.375308+00:00,Allocation/combining_fo_and_mo_scope/strategy,Balanced,Allocation/combining_fo_and_mo_scope/executed_...,150.0,LUID_00003D53


## Check the portfolio holdings with orders

In [55]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=ibor_scope, code=ibor_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True).sort_values(by='strategy(middle_office-SubHoldingKeys)')

,instrument_uid,strategy(middle_office-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,sub_holding_keys,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Figi,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/TradeToPortfolioRate.key,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.unit,transaction.source,transaction.entry_date_time
2,LUID_00003D4V,Balanced,Apple_Nasdaq_AAPL,us_long,middle_office,P,3.500000e+05,350000.0,9.417730e+06,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,LUID_00003D4Z,Balanced,USTreasury_6.875_2025,us_long,middle_office,P,9.844400e+04,98444.0,1.387864e+07,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,LUID_00003D53,Balanced,USTreasury_2.00_2021,us_long,middle_office,P,1.433000e+03,1433.0,1.206314e+07,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
0,LUID_00003D4X,Tech,Amazon_Nasdaq_AMZN,us_long,middle_office,P,5.000000e+03,5000.0,7.750000e+06,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,LUID_00003D4V,Tech,Apple_Nasdaq_AAPL,us_long,middle_office,P,4.956700e+04,49567.0,9.417730e+06,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,LUID_00003D4X,NaN,Amazon_Nasdaq_AMZN,us_long,middle_office,O,4.000000e+03,0.0,-2.480000e+10,USD,-2.480000e+10,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Buy,BBG000BVPXP1,LUID_00003D4X,2021-09-30 17:38:06.045577+00:00,2021-10-03 17:38:06.045577+00:00,4000.0,6200000.00,Price,-2.480000e+10,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 17:38:06.045577+00:00
6,CCY_USD,NaN,UNITED STATES DOLLAR,us_long,middle_office,OC,-2.480000e+10,0.0,2.480000e+10,USD,2.480000e+10,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Buy,BBG000BVPXP1,LUID_00003D4X,2021-09-30 17:38:06.045577+00:00,2021-10-03 17:38:06.045577+00:00,4000.0,6200000.00,Price,-2.480000e+10,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 17:38:06.045577+00:00
7,LUID_00003D4Z,NaN,USTreasury_6.875_2025,us_long,middle_office,O,5.643000e+03,0.0,-4.489289e+09,USD,-4.489289e+09,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Buy,BBG000DQQNJ8,LUID_00003D4Z,2021-09-30 17:38:06.045577+00:00,2021-10-03 17:38:06.045577+00:00,5643.0,795550.14,Price,-4.489289e+09,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 17:38:06.045577+00:00
8,CCY_USD,NaN,UNITED STATES DOLLAR,us_long,middle_office,OC,-4.489289e+09,0.0,4.489289e+09,USD,4.489289e+09,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Buy,BBG000DQQNJ8,LUID_00003D4Z,2021-09-30 17:38:06.045577+00:00,2021-10-03 17:38:06.045577+00:00,5643.0,795550.14,Price,-4.489289e+09,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 17:38:06.045577+00:00
9,LUID_00003D53,NaN,USTreasury_2.00_2021,us_long,middle_office,O,1.215430e+05,0.0,-1.466191e+12,USD,-1.466191e+12,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Buy,BBG00FN3B5K8,LUID_00003D53,2021-09-30 17:38:06.045577+00:00,2021-10-03 17:38:06.045577+00:00,121543.0,12063142.75,Price,-1.466191e+12,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 17:38:06.0